This is the ANS test. 
Click the play button to run all the cells in order.
When you reach the last cell the test should begin!


In [1]:
from IPython.display import display, HTML, Image, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import random

##commented out seeding because we want players to be exposed to different time intervals in different orders so that we can eliminate bias due to familiarity with the test
#random.seed(1)


import ipywidgets as widgets
from IPython.display import display

from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json




In [2]:
#For sending results
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [3]:
#For buttons
event_info = {
    'type': '',
    'description': '',
    'time': -1
}


def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    with ui_events() as ui_poll: ui_poll(999)
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info


def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [4]:
def run_ANS_single(img_file,right_answer,display_time=0.75):

    

    
    empty=Image('empty.jpg',width=900)
    pic=Image(img_file,width=900)


    btn1 = widgets.Button(description="LEFT")
    btn2 = widgets.Button(description="RIGHT")
    
    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)

    panel = widgets.HBox([btn1, btn2])
    
    display(pic)
    display(panel)
    
    start_time = time.time()
    result = wait_for_event(timeout=display_time)
    clear_output(wait=False)

    

    if result['description']=="":

        display(empty)
    
        panel = widgets.HBox([btn1, btn2])
    
        display(panel)


        result = wait_for_event(timeout=3)
    
    end_time = time.time()

    reaction_time = end_time - start_time
    
    if result['description'] == '':
        
        score = 0
    elif result['description'] ==  right_answer.upper():
        
        score = 1
    else:
        score = 0
        
    #time.sleep(1.5)
    clear_output()
    wait_for_event(timeout=1.5)
    
    
    return score, reaction_time

In [5]:
def run_ANS_all():


    #Import pictures and corresponding information
    files = ["10_9yb.jpg", "12_14.jpg","12_14yb.jpg","12_16yb.jpg", "12_9.jpg",   "12_9yb.jpg","14_12.jpg","14_12yb.jpg","16_12.jpg","18_16yb.jpg",    "18_20yb.jpg","20_15yba.jpg","8_6by.jpg","8_9.jpg","9_10bya.jpg",   
             "9_12by.jpg","8_9v2.jpg","8_9v3.jpg","8_9v4.jpg","9_10v2.jpg",   "9_10v3.jpg","9_10v4.jpg","9_10v5.jpg","9_10v6.jpg","9_10v7.jpg",  "9_12v2.jpg","9_12v3.jpg","9_12v4.jpg","9_12v5.jpg","9_12v6.jpg",      
             "9_12v7.jpg","10_9ybv2.jpg","10_9ybv3.jpg","10_9ybv4.jpg","12_9v2.jpg",   "12_9v3.jpg","12_9v4.jpg","12_14ybv2.jpg","12_14ybv3.jpg","12_14ybv4.jpg",   "16_12v2.jpg","16_12v3.jpg","16_12v4.jpg","18_20ybv2.jpg","18_20ybv3.jpg","18_20ybv4.jpg"]
    answers = ["left","right","right","right","left",       "left", "left", "left", "left", "left",     "right","left", "left","right","right",   
               "right","right","right","right","right",   "right","right","right","right","right",     "right","right","right","right","right",
               "right","left","left","left","left",    "left","left","right","right","right",    "left","left","left","right","right","right"]
    LN = [10,12,12,12,12, 12,14,14,16,18, 18,20,8,8,9, 9,8,8,8,9,  9,9,9,9,9, 9,9,9,9,9,  9,10,10,10,12,  12,12,12,12,12,  16,16,16,18,18,18]
    RN = [9,14,14,16,9,  9,12,12,12,16,  20,15,6,9,10,  12,9,9,9,10,  10,10,10,10,10,  12,12,12,12,12,  12,9,9,9,9,  9,9,14,14,14,  12,12,12,20,20,20]

    #Set time intervals to be used in the test 
    time_intervals = [0.5, 1, 1.5]


    #Brief introduction
    print("Welcome to the ANS test")
    print("This test has 3 parts and 46 questions in each part")
    time.sleep(2)

    print(f"Pictures will appear for {time_intervals[0]} seconds, {time_intervals[1]} seconds or {time_intervals[2]} seconds")
    print("Click a button to choose the side with more dots")
    print("Answer within 3 seconds")
    print("Every correct answer chosen with in 3 seconds will score 1")
    time.sleep(2)

    print("The highest possible score you can get in each part is 46.")

    
    print("Let's see how well you can perform!")
    time.sleep(9)

    

    results_dict = {
        'time_interval':[],
        'score_or_not': [],
        'filename': [],
        'nL': [],
        'nR': [],
        'ratio': [],
        'correct': [],
        'reaction_time': []
        
        }
    clear_output(wait=False)



    
    #Shuffle time intervals to eliminate the bias of familiarity on the test
    random.shuffle(time_intervals)
    
    for a in range(0,3):


        print(f"Ready for Part {a+1} of the test?")
        time.sleep(3)
        clear_output(wait=False)
        print("GO!")
        clear_output(wait=False)

        randomlist = random.sample(range(len(files)), len(files))

        total=0
        for i in randomlist:
                
            
            output = run_ANS_single(files[i], answers[i], time_intervals[a]) 
            #print(output)
    
            score = output[0]
            reaction_time = output[1]
            total += score
                
    
            time_interval = time_intervals[a]
            filename = files[i]
            nR = RN[i]
            nL = LN[i]
            ratio = nL/nR
            correct = answers[i]
            
    
            results_dict['time_interval'].append(time_interval)
            results_dict['score_or_not'].append(score)
            results_dict['filename'].append(filename)
            results_dict['nR'].append(nR)
            results_dict['nL'].append(nL)
            results_dict['ratio'].append(ratio)
            results_dict['correct'].append(correct)
            results_dict["reaction_time"].append(reaction_time)
            
                
         
                            
        print(f"You scored {total} out of 46")
    
        print(f"That's Part {a+1} of the test!")
        time.sleep(5)

    
        
        clear_output(wait=False)


    print("Thank you for your time!")
    
    #print(results_dict)
    resultsdf = pd.DataFrame(results_dict)
    result_json = resultsdf.to_json()
    
    return result_json


In [6]:
def consent():
    data_consent_info = """DATA CONSENT INFORMATION:
    
    Please read:
    
    we wish to record your response data
    
    to an anonymised public data repository.
    
    Your data will be used for educational teaching purposes
    
    practising data analysis and visualisation.
    
    Please type yes in the box below if you consent to the upload."""
    
    print(data_consent_info)
    
    result = input("> ")
    result = result.lower()
    
    if result == "yes":
    
        print("Thanks for your participation.")
        
        print("Please contact philip.lewis@ucl.ac.uk")
        
        print("If you have any questions or concerns")
        
        print("regarding the stored results.")
    
    else:
        
        # end code execution by raising an exception
        
        raise(Exception("User did not consent to continue test."))

    clear_output(wait=False)
    
    id_instructions = """
    
    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    
    - two letters based on the initials (first and last name) of a childhood friend
    
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    
    then your unique identifer would be CBTC
    
    """
    
    print(id_instructions)
    
    user_id = input("> ")
    
    print("User entered id:", user_id)

    return user_id

In [7]:
def run_this_please():
    user_id = consent()

    
    time.sleep(1)
    
    print("age")
    age=input(">")
    
    print("gender")
    gender=input(">")
    
    clear_output(wait=False)
    result_json=run_ANS_all()
    
    ## record the basic information & send to the google form later
    data_dict= {
        "user_id": user_id,
        'age': age,
        'gender': gender,
        'result_json':result_json
    }
    
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLScAOUNm1m0Ris9XT7My5fyndrWr_9BSd2DMlUK94EQeuGOkLw/viewform?usp=sf_link"
    send_to_google_form(data_dict, form_url)

This cell below give you an idea of what a question is like.

In [11]:
run_ANS_single("16_12v2.jpg","left",10)

(1, 2.9030730724334717)

Run the next cell to start the full test!

In [12]:
run_this_please()

Thank you for your time!
